Import Statements

In [1]:
import sys
import os
from datetime import datetime, timedelta, timezone
from typing import Union
import pandas as pd

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.data_utils import fetch_batch_raw_data
from src.data_utils import transform_raw_data_into_ts_data
import src.config as config

In [2]:
import sys
import os
from datetime import datetime, timedelta, timezone
from typing import Union
import pandas as pd

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.data_utils import fetch_batch_raw_data
from src.data_utils import transform_raw_data_into_ts_data
import src.config as config

# Determine the current date and time (floored to the hour)
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")
print(f"Current date: {current_date}")

# Define the date range for fetching data (last 29 days)
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)
print(f"Fetching data from: {fetch_data_from}")
print(f"Fetching data to: {fetch_data_to}")

# Fetch historical data (shifted back 16 weeks)
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)
rides.head(5)

# Transform raw data into time series format
ts_data = transform_raw_data_into_ts_data(rides)
ts_data.head(5)
ts_data.info()

# Connect to Hopsworks and upload the data to a new feature group
import hopsworks

# Connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# Connect to the feature store
feature_store = project.get_feature_store()

# Connect to or create a new feature group for recent data
feature_group = feature_store.get_or_create_feature_group(
    name="recent_time_series_hourly_feature_group",
    version=1,
    description="Recent time series data at hourly frequency for inference",
    primary_key=["start_station_name", "pickup_hour"],
    event_time="pickup_hour"
)

# Upload the time series data to the feature group
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Current date: 2025-05-10 08:00:00+00:00
Fetching data from: 2025-04-11 08:00:00+00:00
Fetching data to: 2025-05-10 08:00:00+00:00
File already exists for 2024-12.
Loading data for 2024-12...
Before filtering - 2024-12: 2311171 rows
After duration filter - 2024-12: 2308841 rows
After station filter - 2024-12: 2310051 rows
After date range filter - 2024-12: 2310900 rows
Total records: 2,311,171
Valid records: 2,307,489
Records dropped: 3,682 (0.16%)
Successfully processed data for 2024-12.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2025-01.
Loading data for 2025-01...
Before filtering - 2025-01: 2124475 rows
After duration filter - 2025-01: 2122991 rows
After station filter - 2025-01: 2123911 rows
After date range filter - 2025-01: 2124268 rows
Total records: 2,124,475
Valid records: 2,122,262
Records dropped: 2,213 (0.10%)
Successfully processed data for 2025-01.
Combining all monthly data...
Data loading and processing complete!
Inspecti



To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 04:41:49,752 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1225907


Uploading Dataframe: 100.00% |██████████| Rows 2088/2088 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: recent_time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1225907/jobs/named/recent_time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('recent_time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

Setting current date, and setting timelines for data fetching

In [3]:
# Determine the current date and time (floored to the hour)
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")
print(f"Current date: {current_date}")

# Define the date range for fetching data (last 29 days)
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)
print(f"Fetching data from: {fetch_data_from}")
print(f"Fetching data to: {fetch_data_to}")


Current date: 2025-05-10 08:00:00+00:00
Fetching data from: 2025-04-11 08:00:00+00:00
Fetching data to: 2025-05-10 08:00:00+00:00


Function definition for fetching data in a batch for specified timeline

In [4]:
# Fetch historical data (shifted back 16 weeks)
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)
rides.head(5)

# Transform raw data into time series format
ts_data = transform_raw_data_into_ts_data(rides)
ts_data.head(5)
ts_data.info()

File already exists for 2024-12.
Loading data for 2024-12...
Before filtering - 2024-12: 2311171 rows
After duration filter - 2024-12: 2308841 rows
After station filter - 2024-12: 2310051 rows
After date range filter - 2024-12: 2310900 rows
Total records: 2,311,171
Valid records: 2,307,489
Records dropped: 3,682 (0.16%)
Successfully processed data for 2024-12.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2025-01.
Loading data for 2025-01...
Before filtering - 2025-01: 2124475 rows
After duration filter - 2025-01: 2122991 rows
After station filter - 2025-01: 2123911 rows
After date range filter - 2025-01: 2124268 rows
Total records: 2,124,475
Valid records: 2,122,262
Records dropped: 2,213 (0.10%)
Successfully processed data for 2025-01.
Combining all monthly data...
Data loading and processing complete!
Inspecting rides DataFrame before transformation:
               pickup_datetime                ended_at start_station_name  \
276274 2025

Hopsworks Login and loading feature store

In [5]:
# Connect to Hopsworks and upload the data
import hopsworks

# Connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# Connect to the feature store
feature_store = project.get_feature_store()

# Connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time series data at hourly frequency",
    primary_key=["start_station_name", "pickup_hour"],
    event_time="pickup_hour"
)

2025-05-10 04:42:15,085 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 04:42:15,089 INFO: Initializing external client
2025-05-10 04:42:15,090 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 04:42:15,644 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1225907


In [6]:
# Upload the time series data to the feature group
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 2088/2088 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: citi_bike_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1225907/jobs/named/citi_bike_hourly_feature_group_1_offline_fg_materialization/executions


(Job('citi_bike_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)